In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
method = "molgrid"

n_frags = 25

In [3]:
import open3d as o3d
from rdkit import Chem
import numpy as np

from collections import defaultdict

import tqdm
import os, sys, re

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
sys.path.append("../../")
sys.path.append("../")

In [5]:
from utils import show, transform_and_add_conformer
from score_pcd import fit_and_score
from molgrid_to_pcd import sensaas_color_groups_rgb_molgrid

print(sensaas_color_groups_rgb_molgrid)


def align(mol1, pcd1, pcd2):
    gfit, cfit, hfit, tran = fit_and_score(
        (pcd1, pcd2),
        voxel_size=0.5,
        threshold=0.5,
        color_groups=sensaas_color_groups_rgb_molgrid,
    )
    transform_and_add_conformer(mol1, tran, fromConfId=0, toConfId=1)
    return gfit, cfit, hfit

[[(0.2, 1.0, 1.0), (0.0, 0.0, 1.0), (0.0, 0.0, 1.0), (0.0, 0.0, 1.0), (0.0, 0.0, 1.0), (0.6, 0.0, 0.0), (0.6, 0.0, 0.0), (0.6, 0.0, 0.0), (0.6, 0.6, 0.0), (0.6, 0.6, 0.0)], [(0.3764705882352941, 0.3764705882352941, 0.3764705882352941), (0.3764705882352941, 0.3764705882352941, 0.3764705882352941), (0.7529411764705882, 0.7529411764705882, 0.7529411764705882), (0.7529411764705882, 0.7529411764705882, 0.7529411764705882), (0.6, 0.2980392156862745, 0.0), (0.6, 0.2980392156862745, 0.0)]]


## Load BRD4 Inhibitors

In [6]:
path = "../ligands/BRD4"

In [7]:
files = []
for f in os.listdir(path):
    fname, ext = os.path.splitext(f)
    if (
        ext == ".pcd"
        and method in fname
        and not "tran" in fname
        and not "murcko" in fname
    ):
        files.append(os.path.join(path, f))

# Sort BRD4 ligand files by number
r = re.compile("\d{1,2}")
files.sort(key=lambda f: int(r.search(os.path.basename(f)).group()))

print(files)

pcds = []
mols = []
for f in files:
    pcd = o3d.io.read_point_cloud(f)
    pcds.append(pcd)

    s = Chem.SDMolSupplier(f.replace(".pcd", ".sdf").replace(f"_{method}", ""))
    mol = next(s)
    mols.append(mol)

['../ligands/BRD4/ligand-1_molgrid.pcd', '../ligands/BRD4/ligand-2_molgrid.pcd', '../ligands/BRD4/ligand-3_molgrid.pcd', '../ligands/BRD4/ligand-4_molgrid.pcd', '../ligands/BRD4/ligand-5_molgrid.pcd', '../ligands/BRD4/ligand-6_molgrid.pcd', '../ligands/BRD4/ligand-7_molgrid.pcd', '../ligands/BRD4/ligand-8_molgrid.pcd', '../ligands/BRD4/ligand-9_molgrid.pcd', '../ligands/BRD4/ligand-10_molgrid.pcd']


## Load Fragments

Load fragments. Only fragments that are considered to be synthetically accessible are considered here, in order to reduce computational resources:

In [8]:
fragpath = path = "../../fragments/VEHICLe-good/"
fragfiles = [
    os.path.join(fragpath, f)
    for f in os.listdir(fragpath)
    if os.path.splitext(f)[-1] == ".pcd"
]

fragfiles = fragfiles[:n_frags]

fragpcds = []
fragmols = []
for ff in tqdm.tqdm(fragfiles):
    pcd = o3d.io.read_point_cloud(ff)
    fragpcds.append(pcd)

    s = Chem.SDMolSupplier(ff.replace(".pcd", ".sdf"))
    mol = next(s)
    fragmols.append(mol)

100%|██████████| 25/25 [00:00<00:00, 1131.91it/s]


## Align Fragments to CDK2 Inhibitors

In [9]:
alignments = {}

for i, f in enumerate(files):
    name = os.path.splitext(os.path.basename(f))[0].replace(f"_{method}", "")

    alignments[name] = defaultdict(list)

    for j, ff in enumerate(tqdm.tqdm(fragfiles, desc="Fragments " + name)):
        fidx = os.path.splitext(os.path.basename(ff))[0].replace(f"fragment_", "")

        gfit, cfit, hfit = align(fragmols[j], fragpcds[j], pcds[i])

        alignments[name]["fragment"].append(int(fidx))
        alignments[name]["gfit"].append(gfit.fitness)
        alignments[name]["cfit"].append(cfit.fitness)
        alignments[name]["hfit"].append(hfit)
        alignments[name]["mol"].append(mols[i])
        alignments[name]["fragmol"].append(fragmols[j])

Fragments ligand-10: 100%|██████████| 25/25 [00:07<00:00,  3.24it/s]


In [10]:
import pandas as pd

d = {}
for outerKey, innerDict in alignments.items():
    for innerKey, values in innerDict.items():
        d[(outerKey, innerKey)] = values

df = pd.DataFrame.from_dict(d)
df = df.stack(level=0).swaplevel().sort_index()
df.index.names = ["lig", "idx"]

In [11]:
df["cfit + hfit"] = df["cfit"] + df["hfit"]
df

cfit  fragment  \
lig      idx                       
ligand-1 0    0.867435      1046   
         1    0.941176         0   
         2    0.926380         1   
         3    0.809783      1047   
         4    0.883582        10   
...                ...       ...   
ligand-9 20   0.807786      1006   
         21   0.863636      1055   
         22   0.834568      1007   
         23   0.900568      1056   
         24   0.826471      1008   

                                                       fragmol      gfit  \
lig      idx                                                               
ligand-1 0    <rdkit.Chem.rdchem.Mol object at 0x7fdd75a0bb80>  0.841499   
         1    <rdkit.Chem.rdchem.Mol object at 0x7fdd75a0bbe0>  0.818182   
         2    <rdkit.Chem.rdchem.Mol object at 0x7fdd75a0b940>  0.447853   
         3    <rdkit.Chem.rdchem.Mol object at 0x7fdd75a0b9a0>  0.760870   
         4    <rdkit.Chem.rdchem.Mol object at 0x7fdd75a0bb20>  0.782090   
...                                                        ...       ...   
ligand-9 20   <rdkit.Chem.rdchem.Mol object at 0x7fdd75a0d220>  0.749392   
         21   <rdkit.Chem.rdchem.Mol object at 0x7fdd75a0d280>  0.766667   
         22   <rdkit.Chem.rdchem.Mol object at 0x7fdd75a0d460>  0.765432   
         23   <rdkit.Chem.rdchem.Mol object at 0x7fdd75a0d4c0>  0.698864   
         24   <rdkit.Chem.rdchem.Mol object at 0x7fdd75a0d520>  0.761765   

                  hfit                                               mol  \
lig      idx                                                               
ligand-1 0    0.645533  <rdkit.Chem.rdchem.Mol object at 0x7fdd759fe9a0>   
         1    0.716578  <rdkit.Chem.rdchem.Mol object at 0x7fdd759fe9a0>   
         2    0.625767  <rdkit.Chem.rdchem.Mol object at 0x7fdd759fe9a0>   
         3    0.581522  <rdkit.Chem.rdchem.Mol object at 0x7fdd759fe9a0>   
         4    0.540299  <rdkit.Chem.rdchem.Mol object at 0x7fdd759fe9a0>   
...                ...                                               ...   
ligand-9 20   0.576642  <rdkit.Chem.rdchem.Mol object at 0x7fdd759feb20>   
         21   0.524242  <rdkit.Chem.rdchem.Mol object at 0x7fdd759feb20>   
         22   0.592593  <rdkit.Chem.rdchem.Mol object at 0x7fdd759feb20>   
         23   0.659091  <rdkit.Chem.rdchem.Mol object at 0x7fdd759feb20>   
         24   0.567647  <rdkit.Chem.rdchem.Mol object at 0x7fdd759feb20>   

              cfit + hfit  
lig      idx               
ligand-1 0       1.512968  
         1       1.657754  
         2       1.552147  
         3       1.391304  
         4       1.423881  
...                   ...  
ligand-9 20      1.384428  
         21      1.387879  
         22      1.427160  
         23      1.559659  
         24      1.394118  

[250 rows x 7 columns]

In [12]:
def getbest(df, lig, criteria="cfit", k=3):
    return df.query("lig == @lig").sort_values(criteria, ascending=False).head(k)


def showbest(df, lig, criteria="cfit", k=10):
    best = getbest(df, lig, criteria, k)

    for idx, row in best.iterrows():
        p = show(row.fragmol, row.mol)
        print(row[criteria])
        display(p)

In [13]:
showbest(df, "ligand-7", criteria="cfit + hfit")

1.4919354838709677


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

1.4764705882352942


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

1.4696969696969697


...

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

1.4289340101522843


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

1.4236842105263157


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol